In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import catboost

import gc
from tqdm import tqdm_notebook
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split, GroupKFold
tqdm.pandas()

In [2]:
import random
import os 
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

In [3]:
root_path = "/data/project/recsys-challenge-2023/sharechat_recsys2023_data/"
train_file = os.path.join(root_path, "train/train.parquet")
test_file = os.path.join(root_path, "test/test.parquet")

train_df = pd.read_parquet(train_file)
test_df = pd.read_parquet(test_file)

In [5]:
trn_idx = train_df[train_df["f_1"] != 66].index
val_idx = train_df[train_df["f_1"] == 66].index

valid_df = train_df.iloc[val_idx]
train_df = train_df.iloc[trn_idx]

In [6]:
print(train_df.shape, valid_df.shape)

(3387880, 83) (97972, 83)


In [7]:
# f_7 제외해봐야함 
features = [
    c for c in train_df.columns if c not in [
        'is_clicked', 'is_installed', 'file_name', 'f_0', 'target'
    ]
]
print(features)

['f_1', 'f_2', 'f_3', 'f_4', 'f_5', 'f_6', 'f_7', 'f_8', 'f_9', 'f_10', 'f_11', 'f_12', 'f_13', 'f_14', 'f_15', 'f_16', 'f_17', 'f_18', 'f_19', 'f_20', 'f_21', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_27', 'f_28', 'f_29', 'f_30', 'f_31', 'f_32', 'f_33', 'f_34', 'f_35', 'f_36', 'f_37', 'f_38', 'f_39', 'f_40', 'f_41', 'f_42', 'f_43', 'f_44', 'f_45', 'f_46', 'f_47', 'f_48', 'f_49', 'f_50', 'f_51', 'f_52', 'f_53', 'f_54', 'f_55', 'f_56', 'f_57', 'f_58', 'f_59', 'f_60', 'f_61', 'f_62', 'f_63', 'f_64', 'f_65', 'f_66', 'f_67', 'f_68', 'f_69', 'f_70', 'f_71', 'f_72', 'f_73', 'f_74', 'f_75', 'f_76', 'f_77', 'f_78', 'f_79']


## Adversarial Validation 
- Train <-> Validation 간의 Gap을 최대화 시키는 변수 확인 

In [8]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
lgb_params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc', # log_loss
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
         }

In [13]:
_valid_df = valid_df.copy()
_train_df = train_df.copy()

_train_df["target"] = 0
_valid_df["target"] = 1

total_df = pd.concat([_valid_df, _train_df], axis=0)

In [15]:
adversarial_scores = []
for col in tqdm(features):
    print(f"col : {col}")
    X_train, X_valid, y_train, y_valid = train_test_split(total_df[[col]], total_df["target"], test_size=0.5, random_state=42, shuffle=True)
    trn_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_valid, label=y_valid)

    clf = lgb.train(lgb_params, trn_data, 50, valid_sets = [trn_data, val_data], verbose_eval=-1, early_stopping_rounds=100)
    adversarial_scores += [clf.best_score["valid_1"]["auc"]]
    print("=="*50)

  0%|          | 0/79 [00:00<?, ?it/s]

col : f_1
Training until validation scores don't improve for 100 rounds


  1%|▏         | 1/79 [00:05<07:38,  5.88s/it]

Did not meet early stopping. Best iteration is:
[1]	training's auc: 1	valid_1's auc: 1
col : f_2
Training until validation scores don't improve for 100 rounds


  3%|▎         | 2/79 [00:15<10:05,  7.86s/it]

...

Did not meet early stopping. Best iteration is:
[1]	training's auc: 0.500369	valid_1's auc: 0.5003
col : f_78
Training until validation scores don't improve for 100 rounds


 99%|█████████▊| 78/79 [09:30<00:06,  6.73s/it]

Did not meet early stopping. Best iteration is:
[1]	training's auc: 0.503122	valid_1's auc: 0.502571
col : f_79
Training until validation scores don't improve for 100 rounds


100%|██████████| 79/79 [09:35<00:00,  7.28s/it]

Did not meet early stopping. Best iteration is:
[1]	training's auc: 0.501694	valid_1's auc: 0.501456


In [19]:
# DataFrame 생성 및 점수 확인 
df = pd.DataFrame(adversarial_scores, columns=["adv_score"])
df["feature"] = features

In [21]:
df = df.sort_values(by="adv_score", ascending=False).reset_index(drop=True)
df

,adv_score,feature
0,1.000000,f_1
1,0.992389,f_64
2,0.991851,f_51
3,0.990743,f_43
4,0.990632,f_70
...,...,...
74,0.500000,f_29
75,0.500000,f_28
76,0.500000,f_27
77,0.500000,f_26
